In [9]:
import pandas as pd
import numpy as np
import datetime as dt     
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Function which calculates number of win and lose trades in a row

def win_lose_streak(df: pd.DataFrame)-> float:
    streak = np.where(df["profit"] > 0, 1, -1)
    win = np.where(streak == 1, np.maximum.accumulate(streak), 0)
    lose = np.where(streak == -1, np.minimum.accumulate(streak), 0)
    max_win = win.max()
    max_lose = lose.min()
    average_win_streak = {'average_win_streak': max_win / len(df)}
    average_lose_streak = {'average_lose_streak': max_lose / len(df)}
    max_win =  {'max_wins': max_win}
    max_lose = {'max_lose': max_lose}
    return [max_win, max_lose, average_win_streak, average_lose_streak]


In [112]:
class Load_data:
    """
    Class loads data, preprocess it, make calculations of strategy performance and displays statistics.
    """
    def __init__(self, data):
        """
        Method load data from csv to dataframe
        """
        self.df = pd.read_csv(data)
        self.df["time"] = pd.to_datetime(self.df["time"].apply(lambda x: x.split('+')[0].replace('T', ' ')))
        self.df.dropna(axis = 1, how="all", inplace = True)
        self.df.drop(labels =["Plot.3", "Plot.4"], axis = 1, inplace = True)
        self.df.rename({"Plot":"Upper band", "Plot.1":"Lower band"}, axis = "columns", inplace = True)
        self.df = self.df[(self.df["Long/Buy"].notna()) | (self.df["Short/Sell"].notna())]
        self.df["Avg_%_to_SL"] = np.where(self.df["Long/Buy"].notna(), (self.df["Lower band"] - self.df["close"]) / self.df["close"], np.nan)
        self.df["sell breakout"] = self.df["sell breakout"].shift(-1)
        self.df["profit"] = (self.df["sell breakout"] - self.df["Buy breakout"]) / self.df["Buy breakout"] * 100
        self.df["gross_profit"] = [x if x > 0 else 0 for x in self.df["profit"]]
        self.df["gross_loss"] = [x if x < 0 else 0 for x in self.df["profit"]]
        self.df["time_periods"] = pd.cut(self.df["time"], bins=[pd.to_datetime("2013-12-05"), pd.to_datetime("2015-11-01"), 
                                                                pd.to_datetime("2017-12-16"), pd.to_datetime("2019-01-22"), 
                                                                pd.to_datetime("2021-04-12"), pd.to_datetime(dt.date.today())], 
                                         labels=["2013-2015", "2015-2017", "2018-2019", "2019-2021", "2021-2022"])
        
    def check(self):
        return self.df[["Avg_%_to_SL", "Long/Buy", "Lower band", "close", "Long/Buy"]]
        
    def display_statistics(self, param_to_show = ["Distance_to_SL", "Profit"]):
        if "Distance_to_SL" in param_to_show:
            print("Distance to SL:")
            print("All Time:\n"
                  f"Mean stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[1] * 100, 2)}\n"
                  f"Standard deviation %: {round(self.df['Avg_%_to_SL'].describe()[2] *100, 2)}\n"
                  f"Maximum stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[3] * 100, 2)}\n"
                  f"Minimum stop-loss %: {round(self.df['Avg_%_to_SL'].describe()[7] * 100, 2)}\n")
        
            print("2013-2015:")
            print(self.df[self.df["time_periods"] == "2013-2015"]["Avg_%_to_SL"].describe())
            print("2015-2017:")
            print(self.df[self.df["time_periods"] == "2015-2017"]["Avg_%_to_SL"].describe())
            print("2018-2019:")
            print(self.df[self.df["time_periods"] == "2018-2019"]["Avg_%_to_SL"].describe())
            print("2019-2021:")
            print(self.df[self.df["time_periods"] == "2019-2021"]["Avg_%_to_SL"].describe())
            print("2021-2022:")
            print(self.df[self.df["time_periods"] == "2021-2022"]["Avg_%_to_SL"].describe())

        if "Profit" in param_to_show:
            print("Profit:")
            print("All Time:")
            print(self.df["profit"].describe())
            print("2013-2015:")
            print(self.df[self.df["time_periods"] == "2013-2015"]["profit"].describe())
            print("2015-2017:")
            print(self.df[self.df["time_periods"] == "2015-2017"]["profit"].describe())
            print("2018-2019:")
            print(self.df[self.df["time_periods"] == "2018-2019"]["profit"].describe())
            print("2019-2021:")
            print(self.df[self.df["time_periods"] == "2019-2021"]["profit"].describe())
            print("2021-2022:")
            print(self.df[self.df["time_periods"] == "2021-2022"]["profit"].describe())


In [114]:
data = Load_data(".\ETHBTC_4H.csv")
# data.check()
data = data.display_statistics(param_to_show="Profit")

Profit:
All Time:
count     41.000000
mean      19.874207
std       60.951199
min      -19.661638
25%       -9.475012
50%       -3.329877
75%       21.946980
max      314.707842
Name: profit, dtype: float64
2013-2015:
count    0.0
mean     NaN
std      NaN
min      NaN
25%      NaN
50%      NaN
75%      NaN
max      NaN
Name: profit, dtype: float64
2015-2017:
count     19.000000
mean      30.903027
std       85.000179
min      -19.661638
25%      -11.329851
50%       -5.435129
75%       20.549066
max      314.707842
Name: profit, dtype: float64
2018-2019:
count     4.000000
mean      5.287843
std      18.384658
min     -12.970608
25%      -9.300224
50%       6.012007
75%      20.600073
max      22.097965
Name: profit, dtype: float64
2019-2021:
count     11.000000
mean      16.913889
std       33.641417
min      -10.364261
25%       -5.968036
50%        4.588944
75%       30.876571
max      102.310989
Name: profit, dtype: float64
2021-2022:
count     7.000000
mean      2.925832
std     